In [1]:
#importing the libraries associated with the code and the folder

import torch   #BEST LIBRARY FOR DEVELOPING NUERAL NETWORKS
from torch.autograd import Variable  #MODULE RESPONSIBLE FOR GRADIENT DESCENT AND WEIGHTS OF THE NEUNET
import cv2   #FOR DRAWING RECTANGLES AROUND THE OBJECT
from data import BaseTransform, VOC_CLASSES as labelmap    #DATA IS A FOLDER WHICH HAS THE CLASSES.
from ssd import build_ssd  #WILL CONSTRUCT AN SSD ARCHITECTURE
import imageio   #FOR PROCESSING THE IMAGES IN THE VIDEO

In [2]:
#DEFINE THE DETECT FUCNCTION
def detect(frame, net, transform): #we are just importing original frame from the video. many frames will be imported. net is ssd neunet. transform will transform the images to get into neunet
    
    #first thing we have to do is take the height and width of the image
    height, width = frame.shape[:2]
    #now we have to do several transformations on the image frame. we have 4 transfromations to do
    
    #1. to transfrom the frame into right transfrom so that our input frame as right color and right dimensions.
    frame_t = transform(frame)[0]
    #2. frame_t is a numpy array. we have to convert it into torch tensor. it is a much more advanced matrix.
    x = torch.from_numpy(frame_t).permute(2,0,1) #this converts RBG TO GRB.
    #3,4. to add a fake dimension to the batch. the neunet accepts batches of inputs. step 4 is cvt torch tensor to torch variable 
    x = Variable(x.unsqueeze(0))
    
    #now we have to feed the nuenet with the image and get output y
    y = net(x)
    
    #we need to take a data attribute from y. we want values of the output
    detections = y.data
    scale = torch.Tensor([width, height, width, height]) #upper left corner and lower left. so 4
    
    #now we go into the heart of the ssd
    #the detections tensor contains four elements. [batch_of_outputs, number of classes, number of occurence, tuple of 5 elements(score, x0, Y0, x1, y1)]
    
    #so we have to make a for loop that goes thwough each class
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            pt = (detections[0, i, j, 1:] * scale).numpy()
            #now we add a rectangle to the detected frame. 1st argument is frame. 2nd coordinates of upper left corner. 3rd coordinates of lower right corner
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            #now we have to print a label box
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            #now we have to increment j i.e the next occerence of class i
            j += 1
        #while loop end
    #for loop end
    
    return frame

#creating the ssd neural netwrok
net = build_ssd('test')

#to load the weights from the pretrained neuralnet
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))

#now we need to create the right transfroms that will have the values that are compatible with the ssd neural netwrok
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0)) #net.size will provide the correct size. and the other 3 args will provide the correct colour values

#now we will import the video and do object detection on it.
reader = imageio.get_reader('original.mp4')
fps = reader.get_meta_data()['fps']

#now we have to create the video with the same fps sequence
write = imageio.get_writer('output.mp4', fps = fps)

for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform) #this frame is different from the one in the for loop
    write.append_data(frame)  #frame to be appended in the vdeo output
    print(i)
    
write.close()

C:\Users\Achaleshwari\Desktop\DIGVIJAY\udemy courses\computer vision\Code for Windows\ssd.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)
C:\Users\Achaleshwari\Desktop\DIGVIJAY\udemy courses\computer vision\Code for Windows\layers\modules\l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
C:\Users\Achaleshwari\Desktop\DIGVIJAY\udemy courses\computer vision\Code for Windows\ssd.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.softmax(conf.view(-1, self.num_classes)),  # conf preds
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (E

0


..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:648: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


1


..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:648: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


...

297


..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:648: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


298


..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
..\torch\csrc\autograd\python_function.cpp:648: UserWarning: Legacy autograd function object was called twice.  You will probably get incorrect gradients from this computation, as the saved tensors from the second invocation will clobber the saved tensors from the first invocation.  Please consider rewriting your autograd function in the modern style; for information on the new format, please see: https://pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd


299
